In [ ]:
from __future__ import division, print_function

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline 

import SimpleITK as sitk

import numpy as np
import random

from tf_unet import image_util

from tf_unet import unet

from skimage.transform import resize

In [ ]:
#gpu coding with all available nodes
config = tf.ConfigProto()

config.gpu_options.allow_growth = True
session = tf.Session(config = config)

In [ ]:
#data loading
data_provider = image_util.ImageDataProvider("*.tif")

In [ ]:
net = unet.Unet(channels = data_provider.channels, n_class = data_provider.n_class, layers = 3, features_root = 64)

In [ ]:
#setup & training
trainer = unet.Trainer(net, batch_size = 2, verification_batch_size = 4, optimizer = "momentum", opt_kwargs = dict(momentum = 0.8))

In [ ]:
my_prediction = sitk.ReadImage('image_'+str(n)+'.tif',sitk.sitkFloat32) #read nii file
my_prediction_np = sitk.GetArrayFromImage(my_prediction)  # store data as array
my_prediction_np_re = np.reshape(my_prediction,(1,520,520,1))
my_prediction_np_re -= np.amin(my_prediction_np)
my_prediction_np_re /= np.amax(my_prediction_np)
mymask = sitk.ReadImage('image'+str(n)+'_mask.tif')
mymask = sitk.GetArrayFromImage(mymask)

In [ ]:
#verification
prediction = net.predict("./unet_trained/model.ckpt", my_prediction_np_re)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex = True, sharey = True,figsize=(12,5))
ax0 = fig.add_subplot(131)
ax0.imshow(my_prediction_np, aspect="auto",cmap = plt.cm.Greys_r)
ax1 = fig.add_subplot(132)
ax1.imshow(mymask, aspect = "auto",cmap = plt.cm.Greys_r)
mask = prediction[0,...,1] >0.5
ax2 = fig.add_subplot(133)
ax2.imshow(mask, aspect="auto",cmap = plt.cm.Greys_r)

ax0.set_title("Input")
ax1.set_title("Ground truth")
ax2.set_title("Prediction")
fig.tight_layout()
fig.savefig("./results.png")  